In [45]:
import pandas as pd

COLUMNS_TO_TAKE = {
    "organ": "dimension",
    "view": "subdimension",
    "transformation": "sub_subdimension",
    "architecture": "algorithm",
    "N_all": "sample_size",
    "RMSE_all": "rmse",
    "RMSE_sd_all": "rmse_std",
    "R-Squared_all": "r2",
    "R-Squared_sd_all": "r2_std",
}

DATA_TYPE_NAMING = {"instances": "all_samples", "eids": "average_per_participant"}
ALGORITHMS_NAMING = {"ElasticNet": "elastic_net", "LightGBM": "ligth_gbm", "NeuralNetwork": "neural_network"}


for data_type in ["instances"]:  #["eids", "instances"]:
    scores_raw = pd.read_csv(
        f"../../all_data/page2_predictions/Performances/PERFORMANCES_tuned_alphabetical_{data_type}_Age_test.csv"
    )
    scores_ = scores_raw[COLUMNS_TO_TAKE.keys()].rename(columns=COLUMNS_TO_TAKE)
    scores = scores_.replace(ALGORITHMS_NAMING).replace({"ImmuneSystem": "BloodCells"}).reset_index(drop=True)
    scores.loc[(scores["dimension"] == "Musculoskeletal") & (scores["sub_subdimension"] == "MRI"), "sub_subdimension"] = "DXA"

In [48]:
scores[scores["dimension"] == "Musculoskeletal"]

,dimension,subdimension,sub_subdimension,algorithm,sample_size,rmse,rmse_std,r2,r2_std
156,Musculoskeletal,FullBody,Figure,InceptionResNetV2,42164,3.401060,0.012459,0.797196,0.001718
157,Musculoskeletal,FullBody,Figure,InceptionV3,42164,3.463083,0.012336,0.789732,0.001803
158,Musculoskeletal,FullBody,Flesh,InceptionResNetV2,42165,3.491831,0.012854,0.786221,0.001768
159,Musculoskeletal,FullBody,Flesh,InceptionV3,42165,3.587206,0.013521,0.774384,0.001973
160,Musculoskeletal,FullBody,Mixed,InceptionResNetV2,42164,3.294960,0.011919,0.809652,0.001645
161,Musculoskeletal,FullBody,Mixed,InceptionV3,42164,3.376518,0.012569,0.800112,0.001781
162,Musculoskeletal,FullBody,Skeleton,InceptionResNetV2,42164,3.596605,0.012375,0.773205,0.001810
163,Musculoskeletal,FullBody,Skeleton,InceptionV3,42164,3.648985,0.013319,0.766551,0.001961
164,Musculoskeletal,Hips,DXA,InceptionResNetV2,42146,4.457411,0.015046,0.651699,0.002498
165,Musculoskeletal,Hips,DXA,InceptionV3,42146,4.329807,0.014607,0.671355,0.002476
